# 1. 모델 YOLOV5 다운로드

In [ ]:
%cd /content
!git clone https://github.com/ultralytics/yolov5 
%cd yolov5
%pip install -qr requirements.txt

import torch
from yolov5 import utils
display = utils.notebook_init()  # checks

# 2. pistol 데이터셋을 설치한다.

 - pistol 폴더를 만들어 새로 생긴 화일을 저장한다.

In [ ]:
%mkdir /content/pistol
%cd /content/pistol
!curl -L "https://public.roboflow.com/ds/8DKIFktQF6?key=ySWhdch35X" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

- 이미지(images) 갯수 확인

In [ ]:
%cd /
from glob import glob

img_list=glob('/content/pistol/export/images/*.jpg')

print(len(img_list))

- pistol에 어떤 것들이 들어 있을까?

In [ ]:
!ls /content/pistol

- pistol/export에 무엇이 있을까요?

In [ ]:
!ls /content/pistol/export/

# 3. train 과 val 나누기

- images와 labels 데이터가 train과 val이 구분이 되어있지 않음.



- images와 labels 데이터를 train과 val을 80:20 으로 분류한다.

In [ ]:
from sklearn.model_selection import train_test_split

train_img_list, val_img_list = train_test_split(img_list, test_size=0.2, random_state=2000)

print(len(train_img_list), len(val_img_list))

- data.yaml의 경로를 확인

In [ ]:
%cat /content/pistol/data.yaml

## train 이미지 경로와 val 이미지 경로로 txt파일을 저장

In [ ]:
with open('/content/pistol/train.txt', 'w') as f:
  f.write('\n'.join(train_img_list) + '\n')

with open('/content/pistol/val.txt', 'w') as f:
  f.write('\n'.join(val_img_list) + '\n')  

## 4. 화면의 좌측에서 data.yaml을 수정하거나, 미리 수정한 pistol.yaml을 pistol 아래에 설치한다.

## train: /content/pistol/train.txt
## val: /content/pistol/val.txt

## nc: 1
## names: ['pistol']

## 경로가 잘 수정되었는지 확인한다.

In [ ]:
%cat /content/pistol/data.yaml

# 5. 학습하기

###    gun_results 폴더에 학습한 모델을 저장

- data : 데이터셋의 경로정보를 가진 yaml 화일 설정
- cfg : 학습에 사용할 모델(yolov5s.pt)의 설정과 관련한 화일(yolov5s.yaml) 지정
- weights : 학습에 사용할 모델(yolov5s.pt) 지정

In [ ]:
%%time
%cd /content/yolov5/
!python train.py --img 416 --batch 16 --epochs 10 --data /content/pistol/data.yaml --cfg ./models/yolov5s.yaml --weights yolov5s.pt --name pistol_results --cache

- 모델 생성 결과 확인

In [ ]:
!ls /content/yolov5/runs/train/pistol_results4/weights

# 6. tensorboard 로 학습이 잘 진행되었는지 확인

In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs

# 7. 검증하기(validation)
 val.py 사용

- weights : 검증에 사용할 학습된 모델(best.pt) 설정
- data : 데이터셋의 경로정보
- iou : 실측값과 예측값이 얼마나 겹치는지를 나타내는 척도(iou 0.65 : 0.65이하는 버린다.)


In [ ]:
!python val.py --weights runs/train/pistol_results4/weights/best.pt --data /content/pistol/data.yaml --img 640 --iou 0.65 --half